# Battle Royale Survival Rate
## Math concepts for developers final project
**Author: Viktor Georgiev**  
**Date: May 2025**

### Introduction
Let us face it — everybody has at least once in their life played a game like PUBG, Fortnite, Call of Duty Warzone / Mobile, or something similar. 
In recent times, TV series like *Squid Game* and *Alice in Borderland* have further popularized the idea, which possibly started with the *Hunger Games* book series.

This project simulates a simplified Battle Royale scenario and uses **probability theory**, **stochastic simulation**, and **statistical analysis** to estimate each player’s survival chances. 
The results help answer questions like:

- How does initial skill influence survival?
- How many rounds are typically needed until only one survives?

---

### Project goals
- Model a **simplified Battle Royale** with `N` players.
- Implement **random encounters** between players, with probabilistic outcomes based on skill.
- Use **Monte Carlo simulation** to estimate survival rates.
- Apply **probability**, **combinatorics**, and **basic Markov modeling** to understand outcomes.
- Visualize key results using graphs and plots.
- Explore potential **strategic insights** based on survival analysis.

### Methodology overview
- Define a set of players, each with a skill rating.
- Determine outcome of each encounter using **probabilistic rules**.
- Repeat the process until only one player remains.
- Run thousands of simulations to gather statistics.
- Analyze results using probability theory and visualization.

---

### Tools and Concepts
- Probability and Conditional Probability
- Random Variables and Expected Value
- Monte Carlo Simulation
- Markov Chains (absorbing state modeling)
- Statistical Visualization

---

In [1]:
# Imports
import random

## General rules of the simulator and maths

### 1. Player Attributes: Gender, Age and Background

- Gender: This could affect the base skill (e.g., male players could have a higher average skill based on physicality in combat, while female players may have a different strategy).
- Age: The skill of a player decreases with age, especially above 50. This could be represented as a function where the skill **S** decreases as age **A** increases
$$S_{age}(A) = S_{base} * (1 - 0.05 * max(0, A - 50))$$

This formula represents a 5% decrease in skill for every year above 50. For ages below 50, the skill remains the same.

- Backgrounds: Different backgrounds provide different advantages, such as:
    * Programmer: Better at strategy, with higher chance of survival when hiding or avoiding conflict.
    * MMA Fighter: Higher combat skill, but lower strategy or survival when hiding.
    * Soldier: Balanced in combat and survival.
    * Doctor: Can heal or reduce injuries, improving survival chances.
    * Construction worker: Can use equipment better
    * Others

We can model these advantages by adding background multipliers to the base skill:
$$
S_{\text{background}} = S_{\text{base}} \times M_{\text{background}}
$$

where $M_{background}$ is a multiplier based on the background (e.g. MMA fighter $M_{MMA}$ = 1.2, Doctor $M_{doctor}$ = 1.1)

---


### 2. Strategies

Players can adopt different strategies during the game, which affect their survival probability. We can represent this as a conditional probability:

- **Aggressive strategy**:  Higher risk of death but higher chance of eliminating opponents.
- **Hiding strategy**: Lower risk of death, but lower chance of eliminating opponents.
- **Balanced strategy**: Mix of both

We can use a strategy modifier $M_{strategy}$ that adjusts the survival probability based on the strategy:

$$P_{survival}(strategy) = P_{base} * M_{strategy}$$

where $M_{strategy}$ could take values based on the strategy chosen (e.g., for aggressive: $M_{aggressive}$ = 0.8, hiding $M_{hiding}$ = 0.8)


---

### 3. Injuries and Sickness

Injuries or sickness can decrease a player's survival probability. Let’s define a probability reduction factor $R_{injury}$ or $R_{sickness}$ based on the severity of the condition:

$$P_{survival} = P_{base} * (1 - R_{injury}) * (1 - R_{sickness})$$

where:

$R_{injury}$ and $R_{sickness}$ are factors between 0 and 1, where 1 represents a fatal injury or sickness.

---

### 4. Team Dynamics

When players with similar backgrounds team up, they can boost each other’s survival chances. This can be modeled using a cooperative multiplier $M_{team}$

$$P_{survival | team} = P_{base} * M_{team}$$

where $M_{team}$ > 1 (e.g. $M_{team}$ = 1.2) if the players have compatible backgrounds (e.g., two soldiers teaming up).

---

### 5. Zone shrinking
The shrinking game zone could be modeled as a deterministic function where players outside the zone die or suffer a survival penalty. If the player is outside the zone, their survival probability becomes 0, otherwise:

\[
$P_{survival|zone}$ = 
\begin{cases}
0, & \text{if player is outside the zone} \\
P_{\text{base}}, & \text{if player is inside the zone}
\end{cases}
\]


The shrinking zone itself can be modeled with a time-dependent radius $r(t)$ where $r$ decreases as time progresses:

$$r(t) = r_{0} - k * t$$

where $r_{0}$ is the initial radius and $k$ is the shrink rate

--- 

### 6. Weapons and Items

Weapons found during the game can boost the player's skill or alter the outcome of encounters. We can model the weapon effect as a multiplier:

$$S_{weapon} = S_{base} * M_{weapon}$$

where $M_{weapon}$ is  is a multiplier based on the type of weapon (e.g., a gun might have $M_{weapon}$ = 1.5)


---

### 7.Overall Survival Probability

Combining all factors we get this formula:

$$P_{survival} = P_{base} * M_{age} * M_{background} * M_{strategy} * (1 - R_{injury}) * (1 - R_{sickness}) * M_{team} * M_{weapon} * P_{zone}$$

This formula accounts for all the factors affecting a player’s survival.

---

### 8. Bayesian Inference for Dynamic Updates

- **Context**: As the game progresses, new information (such as injuries, encounters, weapons found) becomes available. We can use **Bayesian Inference** to update the survival probabilities dynamically as new data comes in.
- **How to use it**:
    * Each player has a prior survival probability $P\frac{prior}{survival}$ based on their initial conditions (e.g., age, gender, background).
    * As the game progresses, new observations (such as successful encounters or injuries) can be used to update the survival probability using Bayes’ Theorem:

$$P_{survival|data} = \frac{P_{(data|survival)} * P_{survival}}{P_{data}}$$

This allows the simulation to adjust each player's survival probability as they encounter new challenges or make progress through the game.

---


In [4]:
# Player attributes
# Player class
class Player:
    def __init__(self, name, gender, age, background):
        self.name = name
        self.gender = gender.lower()
        self.age = age
        self.background = background.lower()

        self.base_skill = 1.0

        # Gender modifier
        gender_multipliers = {
            'male': 1.0,
            'female': 0.95
        }
        self.M_gender = gender_multipliers.get(self.gender, 1.0)  # Default to 1.0 if unknown

        # Age penalty: -5% per year over 50
        if self.age > 50:
            self.M_age = 1 - 0.05 * (self.age - 50)
        else:
            self.M_age = 1.0

        # Background multiplier
        background_multipliers = {
            'programmer': 0.9,
            'mma_fighter': 1.2,
            'soldier': 1.1,
            'doctor': 1.1,
            'construction_worker': 1.0,
            'other': 1.0
        }
        self.M_background = background_multipliers.get(self.background, 1.0)

        # Total skill modifier
        self.skill = self.base_skill * self.M_gender * self.M_age * self.M_background

    def __repr__(self):
        return f"{self.name} | Gender: {self.gender} | Age: {self.age} | Background: {self.background} | Skill: {self.skill:.2f}"

        

In [ ]:
# Let us check if the age penalty and background multiplier and gender multiplier are working as expected

for age in range(45, 61, 3):
    p = Player("Test", "male", age, "soldier")
    print(f"Age {age} => Skill: {p.skill:.2f}")
    
backgrounds = ['programmer', 'mma_fighter', 'soldier', 'doctor', 'construction_worker', 'other']
for bg in backgrounds:
    p = Player("Test", "male", 30, bg)
    print(f"Background: {bg} => Skill: {p.skill:.2f}")
    


Age 45 => Skill: 1.10
Age 48 => Skill: 1.10
Age 51 => Skill: 1.04
Age 54 => Skill: 0.88
Age 57 => Skill: 0.71
Age 60 => Skill: 0.55
Background: programmer => Skill: 0.90
Background: mma_fighter => Skill: 1.20
Background: soldier => Skill: 1.10
Background: doctor => Skill: 1.10
Background: construction_worker => Skill: 1.00
Background: other => Skill: 1.00


This clearly shows that the logic is working

In [6]:
# Let us add strategy to the class and check if it is working as expected

class Player:
    def __init__(self, name, gender, age, background, strategy):
        self.name = name
        self.gender = gender.lower()
        self.age = age
        self.background = background.lower()
        self.strategy = strategy.lower()

        self.base_skill = 1.0

        # Gender multiplier
        gender_multipliers = {
            'male': 1.0,
            'female': 0.95
        }
        self.M_gender = gender_multipliers.get(self.gender, 1.0)

        # Age multiplier
        self.M_age = 1 - 0.05 * max(0, self.age - 50)

        # Background multiplier
        background_multipliers = {
            'programmer': 0.9,
            'mma_fighter': 1.2,
            'soldier': 1.1,
            'doctor': 1.1,
            'construction_worker': 1.0,
            'other': 1.0
        }
        self.M_background = background_multipliers.get(self.background, 1.0)

        # Strategy multiplier
        strategy_multipliers = {
            'aggressive': 0.8,
            'hiding': 0.8,
            'balanced': 1.0
        }
        self.M_strategy = strategy_multipliers.get(self.strategy, 1.0)

        # Final skill/survival multiplier
        self.skill = (
            self.base_skill
            * self.M_gender
            * self.M_age
            * self.M_background
            * self.M_strategy
        )

    def __repr__(self):
        return (f"{self.name} | Gender: {self.gender} | Age: {self.age} | "
                f"Background: {self.background} | Strategy: {self.strategy} | "
                f"Skill: {self.skill:.2f}")


In [7]:
p1 = Player("Alex", "male", 28, "soldier", "aggressive")
p2 = Player("Sophie", "female", 34, "doctor", "hiding")
p3 = Player("Jake", "male", 51, "mma_fighter", "balanced")

print(p1)
print(p2)
print(p3)

Alex | Gender: male | Age: 28 | Background: soldier | Strategy: aggressive | Skill: 0.88
Sophie | Gender: female | Age: 34 | Background: doctor | Strategy: hiding | Skill: 0.84
Jake | Gender: male | Age: 51 | Background: mma_fighter | Strategy: balanced | Skill: 1.14


There is a difference in the overall skill, so it looks like it is working as expected

In [ ]:
# 